# Discovery of Higgs Boson
In this assignment, we'll use data directly from the LHC (Large Hadron Collider) along with Monte-Carlo (MC) simulations to claim the discovery of the Higgs Boson! Please make sure to use the seed 6372 for both PyTorch and Numpy.

<b> Due Date: 05/03/2024 11:59pm</b> <br>
<b> Submission: Output this file as both .ipynb and .pdf files and zip them before uploading to Gradescope</b> <br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, kstest
import util

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Important for re-production
torch.manual_seed(6372)
np.random.seed(6372)

# Problem Setup

At the LHC, protons are accelerated to incredible speeds before they are collided. The collision results in sub-atomic particles that go through multiple phases of decay. These final decay products are the observables by the LHC detectors. Thus, the Higgs Boson is not observed directly but indirectly through its decay products. Our job is to conclude the existence of the Higgs Boson by observing such decay products. However, things aren't that simple. There are two obstacles to overcome.

First, data generated directly from LHC collisions is huge (on the order of billions of entries). Thus, there has to be some filtering. The first type of filtering is online filtering. This happens automatically by LHC detectors to discard any events/collisions that aren't of primary importance for study, which reduces the size of data by few orders of magnitude. After online filtering, there comes off-line filtering. This filtering happens through the use of sophisticated knowledge of theoritical physics and is typically conducted by humans. <b>The data we have for this assignment comes after these two stages of filtering and is only ~ 500 entries</b>.

Second, even after such filtering, we have confounding decay processes that produce the same final decay products as the Higgs decay! In other words, it is impossible to conclude the Higgs Boson discovery by only looking at the <i>types</i> of decay particles. We then have to look at other properties like kinetic energy, momentum etc... However, such properties are continuous in nature and observing any one value doesn't say much. The only remaining solution, then, is to study the statistical properties of such observations.

So how does statistics help us "discover" the Higgs Boson? Through hypothesis testing!

Luckily, even with confounding processes producing identical types of decay particles, theoretical physics describes different statistical distributions for products coming from Higgs vs. products coming from background. We will denote distribution of data coming Higgs by $Q$ and distribution of data coming from background by $P_0$. To make things more complicated, there is more than one background process. So $P_0$ can only be discribed as a <i>mixture</i> of multiple background processes. In other words, 
$$
P_0 = \sum_i a_i P_{0, i}
$$
where $P_{0, i}$ is the background distribution generated from process $i$, and $a_i$ is the mixing ratio.

Finally, $Q, P_{0, i}$ are not given to us explicitly. In other words, we don't have closed-form expressions for their CDF/PDF. We can only sample from $Q, P_{0, i}$ through Monte-Carlo simulations. We will use these Monte-Carlo samples to compare the real experimental data recorded at the LHC with what the data should be under the null hypothesis. 

<b>1. Data Loading</b>

(1a) (0.5 pt) Read the experimental data from the LHC. Use <b>load_expr_data()</b> function provided in utils.py. Store the dataframe object in the variable <b>data</b>

(1b) (0.5 pt) Print the size of the data and column names. 

(1c) (0.5 pt) Read the simuated MC data. Use the <b>load_processes()</b> function provided in utils.py. Store the list of dataframe objects in a variable <b>mc_processes</b>. Note that the first element of the of the list is the MC Higgs process. The rest are MC background processes.

(1d) (0.5 pt) Print the number of MC background processes. Print the size of the Higgs MC process as well as each of the MC background processes. 

(1e) (0.5 pt) Choose any MC process and print its column names.

(1f) (0.5 pt) Compare MC column names with those of the experimental data. There should be one additional column for the MC processes. What is the meaning of this column? Why can't we have such column in the experimental data?

<b>2. Data Munging</b>

(2a) (0.5 pt) Below we have defined two lists. <b>object_cols</b> specifies the categorial features of our data. In our case, it describes the <b>PID</b> (Particle ID). <b>irrelevant_cols</b> describes predictors to be discarded.

You task is to one-hot encode the categorial features and discard the irrelevant features. Use <b>encoder, trim</b> from utils.py. Read the documentation carefully. It should be clear enough for you to use them.

In [ ]:
object_cols = ['PID1', 'PID2', 'PID3', 'PID4'] 

# Vanilla
irrelevant_cols = ['Unnamed: 0', 'Run', 'Event', 'Q1', 'Q2', 'Q3', 'Q4', 'mass', 'mass_z1', 'mass_z2']

# your code here

<b>Digression on MC Sampling:</b> Recall from the problem setup above that $P_0$ is a mixture of distributions $P_{0, i}$ each with a ratio $a_i$. These ratios describe the relative frequency of each background process in the mixture. Now, MC sampling doesn't only simulate background/higgs collisions. It also simulates the detection process. Because of that, lots of MC generated events won't be detected. Thus, the number of entries for each mc processes dataframe is only a small fraction of the number of MC events generated for that process. For actual numbers of MC events look at <b>nevt_*</b> in util.py. Moreover, the number of MC events for a given process doesn't necessarily match the <i>expected</i> number of events for that process, which is given by <b>luminosity x cross-section</b> (again, look at util.py). Thus, to accurately represent $P_0, Q$, we have to rescale/re-weigh each of the MC processes. 

(2b) (0.5 pt) Create a list <b>weights</b> containing the weights of each of the MC processes. Use <b>compute_weights()</b> in util.py. 

(2c) (0.5 pt) For each of the MC <i>background</i> processes, calculate the sampling frequencies needed to obtain an accurate sample from $P_0$. The sampling frequencies should add up to one. Store these frequencies in a list <b>P0_frequencies</b>. <br>

<b>Hint: </b> Your answer should depend on the sizes of the MC background processes in <b>mc_processes</b>. Also, first elemnt of <b>weights</b> describes the weight of the Higgs process and is irrelevant here.

(2d) (0.5 pt) Use <b>split</b> in util.py to split the data into three partitions <b>train_processes, sim_processes, synth_processes</b>. <b>train_processes</b> is the training MC data we'll use to learn properties/features about $P_0, Q$ (see below). <b>sim_processes</b> will be used to sample from the background distribution $P_0$ using each of the MC background processes. This will used to perform hypothesis testing. In past years, <b>synth_processes</b> was used for the last phase of the project (which will not be the case this year).

(2e) (0.5 pt) Add a new column <b>weight</b> for each process in <b>train_processes</b>. Use the <b>weights</b> obtained above which correspond one-by-one to the processes in <b>train_processes</b>. These weights will be used to learn about the properties of $Q, P_0$ later-on.

<b>3. Fomulating the problem</b>

Recall that $P_0, Q$ represent the background/Higgs data distributions, respectively. Whatever the ground truth is, the observed experimental data is a mixture of both. More precisely, it has a distribution $\pi Q + (1-\pi) P_0$ for some mixing ratio $\pi \in [0, 1]$. 

(3a) (0.5 pt) If our task is to conclude the discovery of the Higgs Boson through hypothesis testing, describe the null hypothesis in words.

(3b) (0.5 pt) Describe the null/alternative hypotheses mathematically in terms of $\pi$.

<b>4. Obtaining $\hat{h}$</b>

Recall from the Neyman-Pearson lemma, the uniformily most powerful test uses the statistic 

$$
h^*(x) = \mathrm{log}\Big(\frac{Q(x)}{P_0(x)}\Big).
$$

Hence, it is natural to study $h^*$. Note, however, that we don't have a closed-form expression for $P_0(x), Q(x)$; only samples. Thus, we'll use the following result:

Any classifier trained with cross-entropy loss
should approximate $h^∗(x)$. More precisely, define the log-loss as usual

$$
l(y, p) = y \mathrm{log} \Big(\frac{1}{p}\Big) + (1-y) \mathrm{log}\Big(\frac{1}{1-p}\Big), \;\;\; y\in \{0, 1\}, p\in [0, 1].
$$

Now, given $n_0 = \nu_0 n$ samples with $y_i = 0$ and $X_i \sim P_0$ and $n_1 = \nu_1 n$
samples with $X_i \sim Q$ we have as $n = n_0 + n_1 \rightarrow \infty$:

$$
\mathrm{log}\Big(\frac{\hat{f}_n}{1-\hat{f}_n}\Big)\rightarrow \mathrm{log}\Big(\frac{\nu_1}{\nu_0}\Big) + h^*
$$

where $\hat{f}_n$ is the sigmoid output of the classifier.
If we have $\nu_0 = \nu_1$, then we have $\mathrm{log}\Big(\frac{\hat{f}_n}{1-\hat{f}_n}\Big)$ is approximately $h^*$.

To that end, our choice of the classifier is a neural network. Moreover, we'll use the <b>weights</b> computed above to re-weigh the training samples for the reasons described in <b>Digression on MC Sampling</b>.

(4a) (0.5 pt) Combine all processes in <b>train_processes</b> into a single dataframe <b>X</b>. Pop the <b>weight</b> column from <b>X</b> and store it in a variable <b>training_weights</b>. Pop the <b>signal</b> column from <b>X</b> into a variable <b>Y</b>.

(4b) (1 pt) Define a PyTorch <b>Dataset</b> class named <b>MyDataset</b> that indexes the samples <b>X[i], Y[i], training_weights[i]</b>. See https://pytorch.org/tutorials/beginner/basics/data_tutorial.html for tutorial.

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, H, training_weights):
        # TODO
        
    def __len__(self):
        # TODO
    
    def __getitem__(self, idx):
        # TODO
    
dataset = MyDataset(X, Y, training_weights)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

(4c) (1 pt) Define a model <b>MyModel</b> that takes as argument the input dimension and returns a Dense Nueral Network (DNN) with three hidden layers. Each hidden layer has 10 units with ReLU activation. The output layer has a single unit with sigmoid activation. See https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html for tutorial.

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        # TODO
        
    def forward(self, x):
        # TODO
        
model = MyModel(X.shape[1])
# notice that reduction='none'
criterion = nn.BCELoss(reduction='none')
opt = torch.optim.Adam(model.parameters())

(4d) (2 pts) Construct the model, then fit it to <b>X, Y</b> weighted by <b>training_weights</b> for 10 epochs. See https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html for tutorial. Print the loss for each epoch.

<b>Note: </b> Make sure the loss is decreasing and after 10 epochs of training, the loss is less than half the loss in the first epoch.

In [ ]:
for epoch in range(1, 11):
    avg_loss = 0
    for x, y, w in dataloader:
        # TODO
    print(f'epoch = {epoch}, loss = {avg_loss:.2e}')

(4e) (2 pts) Plot the histograms of the predicted probabilities of (1) X with label Y=0, (2) X with label Y=1, and (3) <b>data</b> (the experimental data). Show three histograms in the same plot with transparency=0.2. Remember to show the legends as well.

<b>Hint:</b> transparency can be set by using the argument <b>alpha</b> in the plt.hist function.

(4f) (0.5 pt) What can you guess from looking at the histograms?

From the discussion above, define

$$
\hat{h} := \mathrm{log}\Big(\frac{\hat{f}_n}{1-\hat{f}_n}\Big)
$$

which will act as an approximation of $h^*$.

<b>Evaluating $\hat{h}$</b>

(4g) (1 pt) Argue that $\hat{h}$ can be obtained from the trained DNN above but with replacing the sigmoid activation with a linear activation (i.e. $f(x) = x)$

(4h) (3 pts) Re-write the <b>MyModel</b> class to remove the sigmoid activation. Then, train it on <b>X, Y</b> weighted by <b>training_weights</b> for 10 epochs.

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        # TODO
        
    def forward(self, x):
        # TODO
    
model = MyModel(X.shape[1])
# notice the loss function is different this time
criterion = nn.BCEWithLogitsLoss(reduction='none')
opt = torch.optim.Adam(model.parameters())

In [ ]:
for epoch in range(1, 11):
    avg_loss = 0
    for x, y, w in dataloader:
        # TODO
    print(f'epoch = {epoch}, loss = {avg_loss:.2e}')

(4i) (0.5 pt) Construct a $P_0$ sample of size $10^5$ using the MC background processes and <b>P0_frequencies</b> computed above. Make sure to pop <b>signal</b> columns at the end. <br>
<b>Note: </b> the first entry in <b>sim_processes</b> concerns Higgs and is irrelevant here.

(4j) (0.5 pt) Compute $\hat{h}(x)$ for every $x$ in the $P_0$ sample. Store the result in a numpy array <b>h_sim</b>. Compute $\hat{h}(x)$ for every $x$ in experimental data. Store the result in a numpy array <b>h_data</b>.

<b>5. Statistical Tests</b> 

Instead of just looking at the plot, we want to proof the existence of Higgs boson by using hypothesis testing!

<b>Test 1: CLT Test:</b> The CLT test uses the fact that $\hat{h}(X_i)$ are i.i.d where $X_i$ comes from experimental data. We then assume that $n$ is large enough such that we can use the central limit theorem. 

However, how large of an $n$ we need depends on how close the distribution of $\hat{h}(X_i)$ is to a gaussian to start with. The closer to a Gaussian, the smaller the $n$.

(5a) (0.5 pt) Compute and print the mean and standard deviation of $\hat{h}$ on the $P_0$ sample.

(5b) (0.5 pt) Calculate the Kolmogorov–Smirnov test for goodness of fit of $\hat{h}(X_i)$ under $P_0$ against a gaussian with mean/std being the mean/std of $\hat{h}$ under $P_0$, respectively. Print the p-value.

<b>Hint:</b> feel free to use scipy.stats.norm and scipy.stats.kstest.

(5c) (1 pt) Based on the test result, argue against proceeding with the CLT test.

<b>Test 2: Wilcoxon Sum-Rank Test:</b> We'll be using the Mann–Whitney $U$ test (also denoted by the Wilcoxon Sum-Rank test). You can learn more about it here: https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test. It's a non-parametric test that tries to reject the null hypothesis that the distributions of two populations $X, Y$ are identical for the alternative that one population is stochastically greater than the other. More specifically, given two independent samples $(X_1, \dots, X_n), (Y_1, \dots Y_m)$ it computes the following $U$ statistics
$$
U := \sum_{i=1}^n\sum_{j=1}^m 1_{X_i > Y_j}.
$$
Here, we assume $X_i$ and $Y_j$ are continuous random variables so $\mathbb{P}(X_i = Y_j) = 0, \forall i, j$.
For sample sizes $n, m$ small, exact distribution of $U$ under the null hypothesis is known and tabulated. For large values of $n, m$ a Gaussian approximation is used.

(5d) (1 pt) Argue that we can use a one-sided alternative hypothesis when implementing the Wilcoxon test on $\hat{h}(X_i)$ under experimental data vs. $P_0$. 

The Wilcoxon test in scipy.stats.mannwhitneyu used a Gaussian approximation to compute the p-value, which is justified as we have large values of $n, m$. Let's now explore this approximation, and try to implement the test manually ourselves.

(5e) (1 pt) Compute $\mathbb{E}[U]$.

(5f) (2 pts) Compute $\mathbb{E}[U^2]$<br>
<b>Hint:</b> Expand the square then simply/discuss by cases.

(5g) (2 pts) Compute $\mathrm{Var}(U)$.

(5h) (1 pt) We cannot directly use the Central Limit Theorem to approximate the distribution of $U$ by Gaussian. Why not?

As you can see, it is not so simple to conclude that the Gaussian approximation is valid. However, if you're interested, you can read https://www.tandfonline.com/doi/abs/10.1080/10691898.2010.11889486. In this paper, the authors recommend the approximation by a Gaussian with the mean/variance you computed above.

(5i) (2 pts) Assuming the Gaussian approximation, write a function <b>wilcoxon_test</b> that takes in the two arrays <b>h1, h2</b> and output the $U$-statistic and the $p$-value. Print the $U$-statistic and the $p$-value.

<b>Hint</b>: going over all pairs accross <b>h_data, h_sim</b> is inefficient. For a more efficient implementation, start by sorting the combined values of <b>h_data, h_sim</b>.

In [ ]:
def wilcoxon_test(h1, h2):
    # TODO
    
U, pvalue = wilcoxon_test(h_data, h_sim)
print(f'U = {U}')
print(f'p-value = {pvalue:.2e}')

<b>Test 3: Classification Accuracy Test:</b> Previosly, we compare the distributions of <b>h_sim</b> and <b>h_data</b>, which are both logit outputs of the model. 
What if we instead use the class predictions.
Essentially, we compare the ratio of 0's and 1's predictions based on the classifier for both distributions.
Intuitively, it seems to be a worse test than the ones based on logits because we loss some informations when we threshold the outputs into 0's and 1's.

(5j) (0.5 pt) Get the 0-1 predictions of <b>h_sim</b> and <b>h_data</b> by the model by thresholding the logits at 0. Name the arrays as <b>y_sim</b> and <b>y_data</b>, respectively. Then, print the percentage of 1's in each distributions.

(5k) (0.5 pt) What do the percentages of 1's tell you? Does it make sense?

(5l) (2 pts) Now, perform the <b>classification accuracy test</b> by assuming both distributions are binomial that can be approximated as a Gaussian distribution. Write a function <b>accuracy_test</b> which takes in two arrays <b>y1, y2</b> and output the $p$-value. Print the $p$-value.

<b>Hint</b>: similar to pooled two-sampled t-test, but the variance is based on Binomial distribution.

In [ ]:
def accuracy_test(y1, y2):
    # TODO
    
pvalue = accuracy_test(y_data, y_sim)
print(f'p-value = {pvalue:.2e}')

<b>6. Enhanced Features</b> Recall that we discarded a bunch of features as part of the data trimming process. Three of these features were <b>mass, mass_z1, mass_z2</b>. You can think of these features as "smart features" that are engineered/computed through the use of physics knowledge by smart humans to help differentiate Higgs from background. We are now going to explore the effects of using these features in addition to the ones we used previously.

(6a) (26 pts) Repeat the process and calculate the $p$-values of the Wilcoxon test and the accuracy test without discarding the "smart features" <b>mass, mass_z1, mass_z2</b>. Print the $p$-values.

(6b) (0.5 pt) Typically, the standard for declaring "new particle" is 5 sigmas. Here, with the help of "smart features," have we achieve such standard?

However, note that this project is a simplied and scaled-down version of the original experiment. We only have 1 Higgs process and 6 background processes whereas the real-world one has 100 Higgs processes and 1000 background processes. Thus, it is much difficult to achieve 5 sigmas in that case.

<b>7. Discussion</b>

It would be wrong to select what hypothesis test to use <i>after</i> we look at the test results. Ideally, we should be able to choose hypothesis tests before we look at the data given that the assumptions supporting the test hold.

(7a) (1 pt) Ignoring the test performance, what assumptions does the CLT test rely on?

(7b) (1 pt) Why didn't we proceed using the CLT test?

(7c) (1 pt) Ignoring the test performance, what assumptions does the Wilcoxon test and the accuracy test rely on?

 (7d) (1 pt) Are you confident in using the Wilcoxon test or the accuracy test?

(7e) (1 pt) Using the results of the test you selected above, did the "smart features" enhance the detection capability? Why do you think so?

(7f) (1 pt) Comparing the results of the Wilcoxon test and the accuracy test. What do you notice?

In fact, proven in some papers, although thresholding to 0's and 1's, the accuracy test can achieve the same optimal  detection capability compared to other tests that use the logits (or the probabilities).

Congrats on finishing Discovery of Higgs Boson!